In [1]:
from sqlalchemy import create_engine, inspect
import creds, boto3
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn import utils
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
import numpy as np
from matplotlib import pyplot as plt

In [27]:
df = pd.read_csv('products.csv', lineterminator='\n')
del df[df.columns[0]]
df.drop(['create_time\r', 'page_id'], axis=1, inplace=True)

In [3]:
df

,id,product_name,category,product_description,price,location,url
0,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",£5.00,"Wokingham, Berkshire",https://www.gumtree.com/p/mirrors-clocks-ornam...
1,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,£20.00,"Inverness, Highland",https://www.gumtree.com/p/other-household-good...
2,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,£20.00,"Skegness, Lincolnshire",https://www.gumtree.com/p/outdoor-settings-fur...
3,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,£115.00,"Radstock, Somerset",https://www.gumtree.com/p/other-dining-living-...
4,16dbc860-696e-4cda-93f6-4dd4926573fb,Modern Shannon Sofa for sale at low cost | in ...,"Home & Garden / Dining, Living Room Furniture ...",New Design Shannon Corner sofa 5 Seater Avail...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
11313,c4148656-78a9-4f3e-b393-134fdc5ef900,Sony PlayStation VR Move Bundle | in Acocks Gr...,Video Games & Consoles / Consoles / PS4 (Sony ...,Sony PlayStation VR Move Bundle353CASH ON COLL...,£260.00,"Acocks Green, West Midlands",https://www.gumtree.com/p/ps4/sony-playstation...
11314,564e3411-768d-4250-a624-b119d696f103,"Playstation VR V2 Bundle | in Acocks Green, We...",Video Games & Consoles / Consoles / PS4 (Sony ...,Playstation VR V2 Bundle355CASH ON COLLECTION ...,£235.00,"Acocks Green, West Midlands",https://www.gumtree.com/p/ps4/playstation-vr-v...
11315,2b0a652b-46a2-4297-b619-5efeeb222787,"Oculus quest 2 256gb | in Montrose, Angus | Gu...",Video Games & Consoles / Other Video Games & C...,Pick up only £250Comes with two pistols stocks...,£250.00,"Montrose, Angus",https://www.gumtree.com/p/other-video-games/oc...
11316,719fd40a-870e-4144-b324-55dff2e66fb4,Logitech driving force shifter | in Carrickfer...,Video Games & Consoles / Video Game Accessorie...,Bought at christmas from currys retailing at £...,£30.00,"Carrickfergus, County Antrim",https://www.gumtree.com/p/other-video-game-acc...


In [ ]:
## If you want to connect to a SQL database
# host = creds.host
# password = creds.password
# port = creds.port
# user = creds.user
# database = creds.database
# api_type = creds.api_type
# engine = create_engine(f'postgresql+{api_type}://{user}:{password}@{host}:{port}/{database}')
# engine.connect()

In [17]:
cvec = CountVectorizer()
lab_enc = preprocessing.LabelEncoder()

In [4]:
### Cleaning up the price column in the dataframe.
# Removes all rows which have "N/A" in the price column.
df = df.dropna()
# df = df[df['product_name'] != 'N/A']
# df['price'] = df['price'].str.strip('£')
# df['price'] = df['price'].str.replace(',', '')
# df['price'] = df['price'].astype('float64')
# encoded = lab_enc.fit_transform(df['price'])

In [5]:
df

,id,product_name,category,product_description,price,location,url
0,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",£5.00,"Wokingham, Berkshire",https://www.gumtree.com/p/mirrors-clocks-ornam...
1,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,£20.00,"Inverness, Highland",https://www.gumtree.com/p/other-household-good...
2,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,£20.00,"Skegness, Lincolnshire",https://www.gumtree.com/p/outdoor-settings-fur...
3,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,£115.00,"Radstock, Somerset",https://www.gumtree.com/p/other-dining-living-...
12,5707be4f-49d9-4feb-b2c8-aa0868405c65,"Spotlights | in Newent, Gloucestershire | Gumtree",Home & Garden / Other Household Goods,New in box not needed Spotlights . Posted by s...,£9.00,"Newent, Gloucestershire",https://www.gumtree.com/p/other-household-good...
...,...,...,...,...,...,...,...
11313,c4148656-78a9-4f3e-b393-134fdc5ef900,Sony PlayStation VR Move Bundle | in Acocks Gr...,Video Games & Consoles / Consoles / PS4 (Sony ...,Sony PlayStation VR Move Bundle353CASH ON COLL...,£260.00,"Acocks Green, West Midlands",https://www.gumtree.com/p/ps4/sony-playstation...
11314,564e3411-768d-4250-a624-b119d696f103,"Playstation VR V2 Bundle | in Acocks Green, We...",Video Games & Consoles / Consoles / PS4 (Sony ...,Playstation VR V2 Bundle355CASH ON COLLECTION ...,£235.00,"Acocks Green, West Midlands",https://www.gumtree.com/p/ps4/playstation-vr-v...
11315,2b0a652b-46a2-4297-b619-5efeeb222787,"Oculus quest 2 256gb | in Montrose, Angus | Gu...",Video Games & Consoles / Other Video Games & C...,Pick up only £250Comes with two pistols stocks...,£250.00,"Montrose, Angus",https://www.gumtree.com/p/other-video-games/oc...
11316,719fd40a-870e-4144-b324-55dff2e66fb4,Logitech driving force shifter | in Carrickfer...,Video Games & Consoles / Video Game Accessorie...,Bought at christmas from currys retailing at £...,£30.00,"Carrickfergus, County Antrim",https://www.gumtree.com/p/other-video-game-acc...


In [25]:
df['price'] = df['price'].str.strip('£')
# str.strip('£')

/tmp/ipykernel_10751/1632840026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df['price'].str.strip('£')


In [26]:
df

,id,product_name,category,product_description,price,location,url
0,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",,"Wokingham, Berkshire",https://www.gumtree.com/p/mirrors-clocks-ornam...
1,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,,"Inverness, Highland",https://www.gumtree.com/p/other-household-good...
2,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,,"Skegness, Lincolnshire",https://www.gumtree.com/p/outdoor-settings-fur...
3,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,,"Radstock, Somerset",https://www.gumtree.com/p/other-dining-living-...
12,5707be4f-49d9-4feb-b2c8-aa0868405c65,"Spotlights | in Newent, Gloucestershire | Gumtree",Home & Garden / Other Household Goods,New in box not needed Spotlights . Posted by s...,,"Newent, Gloucestershire",https://www.gumtree.com/p/other-household-good...
...,...,...,...,...,...,...,...
11313,c4148656-78a9-4f3e-b393-134fdc5ef900,Sony PlayStation VR Move Bundle | in Acocks Gr...,Video Games & Consoles / Consoles / PS4 (Sony ...,Sony PlayStation VR Move Bundle353CASH ON COLL...,,"Acocks Green, West Midlands",https://www.gumtree.com/p/ps4/sony-playstation...
11314,564e3411-768d-4250-a624-b119d696f103,"Playstation VR V2 Bundle | in Acocks Green, We...",Video Games & Consoles / Consoles / PS4 (Sony ...,Playstation VR V2 Bundle355CASH ON COLLECTION ...,,"Acocks Green, West Midlands",https://www.gumtree.com/p/ps4/playstation-vr-v...
11315,2b0a652b-46a2-4297-b619-5efeeb222787,"Oculus quest 2 256gb | in Montrose, Angus | Gu...",Video Games & Consoles / Other Video Games & C...,Pick up only £250Comes with two pistols stocks...,,"Montrose, Angus",https://www.gumtree.com/p/other-video-games/oc...
11316,719fd40a-870e-4144-b324-55dff2e66fb4,Logitech driving force shifter | in Carrickfer...,Video Games & Consoles / Video Game Accessorie...,Bought at christmas from currys retailing at £...,,"Carrickfergus, County Antrim",https://www.gumtree.com/p/other-video-game-acc...


In [12]:
df['price'] = df.loc[:, ('price')].str.strip('£')

/tmp/ipykernel_10751/1155379902.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df.loc[:, ('price')].str.strip('£')


In [14]:
df.loc[:, ('price')] = df.loc[:, ('price')].str.strip('£')

/tmp/ipykernel_10751/3918964336.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, ('price')] = df.loc[:, ('price')].str.strip('£')


In [6]:
df

,id,product_name,category,product_description,price,location,url
0,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",5.00,"Wokingham, Berkshire",https://www.gumtree.com/p/mirrors-clocks-ornam...
1,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,20.00,"Inverness, Highland",https://www.gumtree.com/p/other-household-good...
2,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,20.00,"Skegness, Lincolnshire",https://www.gumtree.com/p/outdoor-settings-fur...
3,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,115.00,"Radstock, Somerset",https://www.gumtree.com/p/other-dining-living-...
12,5707be4f-49d9-4feb-b2c8-aa0868405c65,"Spotlights | in Newent, Gloucestershire | Gumtree",Home & Garden / Other Household Goods,New in box not needed Spotlights . Posted by s...,9.00,"Newent, Gloucestershire",https://www.gumtree.com/p/other-household-good...
...,...,...,...,...,...,...,...
11313,c4148656-78a9-4f3e-b393-134fdc5ef900,Sony PlayStation VR Move Bundle | in Acocks Gr...,Video Games & Consoles / Consoles / PS4 (Sony ...,Sony PlayStation VR Move Bundle353CASH ON COLL...,260.00,"Acocks Green, West Midlands",https://www.gumtree.com/p/ps4/sony-playstation...
11314,564e3411-768d-4250-a624-b119d696f103,"Playstation VR V2 Bundle | in Acocks Green, We...",Video Games & Consoles / Consoles / PS4 (Sony ...,Playstation VR V2 Bundle355CASH ON COLLECTION ...,235.00,"Acocks Green, West Midlands",https://www.gumtree.com/p/ps4/playstation-vr-v...
11315,2b0a652b-46a2-4297-b619-5efeeb222787,"Oculus quest 2 256gb | in Montrose, Angus | Gu...",Video Games & Consoles / Other Video Games & C...,Pick up only £250Comes with two pistols stocks...,250.00,"Montrose, Angus",https://www.gumtree.com/p/other-video-games/oc...
11316,719fd40a-870e-4144-b324-55dff2e66fb4,Logitech driving force shifter | in Carrickfer...,Video Games & Consoles / Video Game Accessorie...,Bought at christmas from currys retailing at £...,30.00,"Carrickfergus, County Antrim",https://www.gumtree.com/p/other-video-game-acc...


In [55]:
X = df['product_name']
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print("Number of samples in:")
print(f"    Training: {len(y_train)}")
print(f"    Testing: {len(y_test)}")
# min_df=2 will remove all terms that only appear once in the entire data sheet. I assume them to be outliers or words that are never seen again.
# cvec will transform all words into lowercase by default.
# We only fit on our X_train.
# We want to train the model on just the training data and then all we want to do with the testing data is to transform it based on our training data. 
# We still need both of these to be dataframes so we convert our matrices made in the fit to dense matrices so they can be made into dataframes easily.
cvec = CountVectorizer(min_df=2).fit(X_train)
df_product_name_train = pd.DataFrame(cvec.transform(X_train).todense(),columns=cvec.get_feature_names_out())
df_product_name_test = pd.DataFrame(cvec.transform(X_test).todense(),columns=cvec.get_feature_names_out())
print(df_product_name_train.shape)
print(y_train.shape)
print(df_product_name_test.shape)
print(y_test.shape)

Number of samples in:
    Training: 4228
    Testing: 2083
(4228, 3412)
(4228,)
(2083, 3412)
(2083,)


In [56]:
df['price'].describe()

count    6.311000e+03
mean     2.501585e+03
std      1.286437e+05
min      0.000000e+00
25%      1.000000e+01
50%      4.500000e+01
75%      1.500000e+02
max      1.000000e+07
Name: price, dtype: float64

In [57]:
len(df[(df['price'] > 10000)])

31

In [58]:
# removes all rows that have a price value of over 10000
df.drop(df[(df['price'] > 10000)].index, inplace=True)
# removes all rows that have a price of 0
df.drop(df[(df['price'] == 0)].index, inplace=True)


In [59]:
df['price'].describe()

count     6203.000000
mean       195.339258
std        540.488281
min          0.010000
25%         12.000000
50%         45.000000
75%        150.000000
max      10000.000000
Name: price, dtype: float64

In [30]:
df

,id,product_name,category,product_description,price,location,url
0,243809c0-9cfc-4486-ad12-3b7a16605ba9,"Mirror wall art | in Wokingham, Berkshire | Gu...","Home & Garden / Dining, Living Room Furniture ...","Mirror wall art. Posted by Nisha in Dining, Li...",5.0,"Wokingham, Berkshire",https://www.gumtree.com/p/mirrors-clocks-ornam...
1,1c58d3f9-8b93-47ea-9415-204fcc2a22e6,"Stainless Steel Food Steamer | in Inverness, H...",Home & Garden / Other Household Goods,Morphy Richard’s (model no 48755)Stainless ste...,20.0,"Inverness, Highland",https://www.gumtree.com/p/other-household-good...
2,860673f1-57f6-47ba-8d2f-13f9e05b8f9a,"Sun loungers | in Skegness, Lincolnshire | Gum...",Home & Garden / Garden & Patio / Outdoor Setti...,I have 2 of these - collection only as I don’t...,20.0,"Skegness, Lincolnshire",https://www.gumtree.com/p/outdoor-settings-fur...
3,59948726-29be-4b35-ade5-bb2fd7331856,Coffee side table from Ammunition ammo box hai...,"Home & Garden / Dining, Living Room Furniture ...",Great reclaimed army ammunition box used as co...,115.0,"Radstock, Somerset",https://www.gumtree.com/p/other-dining-living-...
12,5707be4f-49d9-4feb-b2c8-aa0868405c65,"Spotlights | in Newent, Gloucestershire | Gumtree",Home & Garden / Other Household Goods,New in box not needed Spotlights . Posted by s...,9.0,"Newent, Gloucestershire",https://www.gumtree.com/p/other-household-good...
...,...,...,...,...,...,...,...
11313,c4148656-78a9-4f3e-b393-134fdc5ef900,Sony PlayStation VR Move Bundle | in Acocks Gr...,Video Games & Consoles / Consoles / PS4 (Sony ...,Sony PlayStation VR Move Bundle353CASH ON COLL...,260.0,"Acocks Green, West Midlands",https://www.gumtree.com/p/ps4/sony-playstation...
11314,564e3411-768d-4250-a624-b119d696f103,"Playstation VR V2 Bundle | in Acocks Green, We...",Video Games & Consoles / Consoles / PS4 (Sony ...,Playstation VR V2 Bundle355CASH ON COLLECTION ...,235.0,"Acocks Green, West Midlands",https://www.gumtree.com/p/ps4/playstation-vr-v...
11315,2b0a652b-46a2-4297-b619-5efeeb222787,"Oculus quest 2 256gb | in Montrose, Angus | Gu...",Video Games & Consoles / Other Video Games & C...,Pick up only £250Comes with two pistols stocks...,250.0,"Montrose, Angus",https://www.gumtree.com/p/other-video-games/oc...
11316,719fd40a-870e-4144-b324-55dff2e66fb4,Logitech driving force shifter | in Carrickfer...,Video Games & Consoles / Video Game Accessorie...,Bought at christmas from currys retailing at £...,30.0,"Carrickfergus, County Antrim",https://www.gumtree.com/p/other-video-game-acc...


In [60]:
lr = LogisticRegression()
lr.fit(df_product_name_train, y_train)
lr.score(df_product_name_test, y_test)

ValueError: Unknown label type: 'continuous'

In [27]:
X = df['location']
y = encoded
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print("Number of samples in:")
print(f"    Training: {len(y_train)}")
print(f"    Testing: {len(y_test)}")
cvec = CountVectorizer(stop_words='english').fit(X_train)
df_location_train = pd.DataFrame(cvec.transform(X_train).todense(),columns=cvec.get_feature_names_out())
df_location_test = pd.DataFrame(cvec.transform(X_test).todense(),columns=cvec.get_feature_names_out())
print(df_location_train.shape)
print(y_train.shape)
print(df_location_test.shape)
print(y_test.shape)

Number of samples in:
    Training: 4794
    Testing: 2362
(4794, 1451)
(4794,)
(2362, 1451)
(2362,)


In [28]:
lr = LogisticRegression()
lr.fit(df_location_train, y_train)
lr.score(df_location_test, y_test)

0.09906858594411516

In [29]:
X = df['product_description']
y = encoded
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print("Number of samples in:")
print(f"    Training: {len(y_train)}")
print(f"    Testing: {len(y_test)}")
cvec = CountVectorizer(min_df=2).fit(X_train)
df_product_description_train = pd.DataFrame(cvec.transform(X_train).todense(),columns=cvec.get_feature_names_out())
df_product_description_test = pd.DataFrame(cvec.transform(X_test).todense(),columns=cvec.get_feature_names_out())
print(df_product_description_train.shape)
print(y_train.shape)
print(df_product_description_test.shape)
print(y_test.shape)

Number of samples in:
    Training: 4794
    Testing: 2362
(4794, 9079)
(4794,)
(2362, 9079)
(2362,)


In [30]:
lr = LogisticRegression()
lr.fit(df_product_description_train, y_train)
lr.score(df_product_description_test, y_test)

/home/oliver/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.15453005927180355

In [31]:
train = pd.concat([df_product_name_train, df_location_train, df_product_description_train], axis=1)
test = pd.concat([df_product_name_test, df_location_test, df_product_description_test], axis=1)
print(train.shape)
print(y_train.shape)
print(test.shape)
print(y_test.shape)


(4794, 14261)
(4794,)
(2362, 14261)
(2362,)


In [32]:
lr = LogisticRegression()
lr.fit(train, y_train)
lr.score(test, y_test)
# lr.predict(test)

/home/oliver/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.14479254868755292

In [ ]:
# np.random.seed(2)

# models = [
#     DecisionTreeRegressor(splitter="random"),
#     SVR(),
#     LinearRegression()
#         ]

# for model in models:
#     model.fit(X_train, y_train)

#     y_train_pred = model.predict(X_train)
#     y_validation_pred = model.predict(X_validation)
#     y_test_pred = model.predict(X_test)

#     train_loss = mean_squared_error(y_train, y_train_pred)
#     validation_loss = mean_squared_error(y_validation, y_validation_pred)
#     test_loss = mean_squared_error(y_test, y_test_pred)

#     print(
#         f"{model.__class__.__name__}: "
#         f"Train Loss: {train_loss} | Validation Loss: {validation_loss} | "
#         f"Test Loss: {test_loss}"
#         )